In [12]:
import requests
import pandas as pd
import json
from legcop import LegiScan

legiscan_api_key='c8d73b2173b9881d495a8a05d59d860d'
legis = LegiScan(legiscan_api_key)

In [8]:
state='ca'
ca_sessions = legis.get_session_list(state=state)
print(ca_sessions)

[{'session_id': 2016, 'state_id': 5, 'year_start': 2023, 'year_end': 2024, 'prefile': 0, 'sine_die': 0, 'prior': 0, 'special': 0, 'session_tag': 'Regular Session', 'session_title': '2023-2024 Regular Session', 'session_name': '2023-2024 Session', 'dataset_hash': '89d69e51e88506408d80690952568b18', 'session_hash': '89d69e51e88506408d80690952568b18', 'name': '2023-2024 Regular Session'}, {'session_id': 1791, 'state_id': 5, 'year_start': 2021, 'year_end': 2022, 'prefile': 0, 'sine_die': 1, 'prior': 1, 'special': 0, 'session_tag': 'Regular Session', 'session_title': '2021-2022 Regular Session', 'session_name': '2021-2022 Session', 'dataset_hash': '48ae3222e08e6cd730ef7c818d467561', 'session_hash': '48ae3222e08e6cd730ef7c818d467561', 'name': '2021-2022 Regular Session'}, {'session_id': 1624, 'state_id': 5, 'year_start': 2019, 'year_end': 2020, 'prefile': 0, 'sine_die': 1, 'prior': 1, 'special': 0, 'session_tag': 'Regular Session', 'session_title': '2019-2020 Regular Session', 'session_name'

In [9]:
start_most_recent_session = ca_sessions[0]['year_start']

datasetlist = legis.get_dataset_list(state=state,
                                     year=start_most_recent_session)

access_key = datasetlist[0]['access_key']
session_id =  datasetlist[0]['session_id']

In [11]:
dataset = legis.get_dataset(session_id=session_id, access_key=access_key)
assert dataset['status']=='OK'
readable_dataset = legis.recode_zipfile(dataset)

In [22]:
namelist = readable_dataset.namelist()

item=namelist[0]
content = readable_dataset.read(item)
content = content.decode('UTF-8')
content = json.loads(content)

for key, value in content['bill'].items():
    print('{}: {}'.format(key, value))

bill_id: 1636377
change_hash: 5a9bcc2c85365baf6186543d316f5116
session_id: 2016
session: {'session_id': 2016, 'state_id': 5, 'year_start': 2023, 'year_end': 2024, 'prefile': 0, 'sine_die': 0, 'prior': 0, 'special': 0, 'session_tag': 'Regular Session', 'session_title': '2023-2024 Regular Session', 'session_name': '2023-2024 Session'}
url: https://legiscan.com/CA/bill/AB1/2023
state_link: https://leginfo.legislature.ca.gov/faces/billStatusClient.xhtml?bill_id=202320240AB1
completed: 0
status: 2
status_date: 2023-05-25
progress: [{'date': '2022-12-05', 'event': 1}, {'date': '2023-01-26', 'event': 9}, {'date': '2023-04-17', 'event': 10}, {'date': '2023-04-18', 'event': 9}, {'date': '2023-04-26', 'event': 10}, {'date': '2023-04-26', 'event': 9}, {'date': '2023-05-10', 'event': 9}, {'date': '2023-05-18', 'event': 10}, {'date': '2023-05-25', 'event': 2}, {'date': '2023-05-26', 'event': 9}, {'date': '2023-06-14', 'event': 9}]
state: CA
state_id: 5
bill_number: AB1
bill_type: B
bill_type_id: 1


Going to want the following:
  url, completed (bool?), status (enum?), status_date, state, bill_number, bill_type(?), title, description, 

In [31]:
test = legis.get_bill(bill_id=19)
df = pd.json_normalize(test)
df.to_excel('excel_test.xlsx')

In [29]:
df.columns

Index(['bill_id', 'change_hash', 'session_id', 'url', 'state_link',
       'completed', 'status', 'status_date', 'progress', 'state', 'state_id',
       'bill_number', 'bill_type', 'bill_type_id', 'body', 'body_id',
       'current_body', 'current_body_id', 'title', 'description',
       'pending_committee_id', 'referrals', 'history', 'sponsors', 'sasts',
       'subjects', 'texts', 'votes', 'amendments', 'supplements', 'calendar',
       'session.session_id', 'session.state_id', 'session.year_start',
       'session.year_end', 'session.prefile', 'session.sine_die',
       'session.prior', 'session.special', 'session.session_tag',
       'session.session_title', 'session.session_name',
       'committee.committee_id', 'committee.chamber', 'committee.chamber_id',
       'committee.name'],
      dtype='object')

In [33]:
master_list = legis.get_master_list()
print(master_list)

ValueError: Must specify session identifier or state. Do not specify both. If you do only the state will be used in the resulting query